# Hello MLlib

In [1]:
# Prereqquisites
from pyspark.sql import SparkSession

In [2]:
# Spark Session and Context
spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext
print("Spark Version: ", spark.version)

Spark Version:  3.4.1


### Data Ingestion

In [4]:
file_path = "data/sf-airbnb/sf-airbnb-clean.parquet/"

df_airbnb = spark.read.parquet(file_path)

df_airbnb.select("neighbourhood_cleansed", "room_type", "bedrooms", "bathrooms", "number_of_reviews", "price").show(5)

+----------------------+---------------+--------+---------+-----------------+-----+
|neighbourhood_cleansed|      room_type|bedrooms|bathrooms|number_of_reviews|price|
+----------------------+---------------+--------+---------+-----------------+-----+
|      Western Addition|Entire home/apt|     1.0|      1.0|            180.0|170.0|
|        Bernal Heights|Entire home/apt|     2.0|      1.0|            111.0|235.0|
|        Haight Ashbury|   Private room|     1.0|      4.0|             17.0| 65.0|
|        Haight Ashbury|   Private room|     1.0|      4.0|              8.0| 65.0|
|      Western Addition|Entire home/apt|     2.0|      1.5|             27.0|785.0|
+----------------------+---------------+--------+---------+-----------------+-----+
only showing top 5 rows

